<a href="https://colab.research.google.com/github/xiborxu/IST462xibo/blob/main/Copy_of_checkpoint2_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/Crash_Reporting_-_Drivers_Data-2.csv')

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
columns_drop = ['Road Name','Cross-Street Name','Report Number', 'Local Case Number', 'Off-Road Description', 'Municipality', 'Related Non-Motorist','Drivers License State', 'Non-Motorist Substance Abuse', 'Person ID', 'Circumstance','Vehicle ID']
# choose the columns we believe is useless
df = df.drop(columns_drop, axis=1)
#drop the columns

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
columns_to_fill_UNKNOWN = ['Driver Substance Abuse', 'Traffic Control','Weather','Equipment Problems','Surface Condition','Collision Type','Light','Vehicle First Impact Location','Vehicle Second Impact Location','Vehicle Movement','Vehicle Make','Vehicle Model','Vehicle Body Type']
df[columns_to_fill_UNKNOWN] = df[columns_to_fill_UNKNOWN].fillna('UNKNOWN')

columns_to_fill_unknown = ['Vehicle Continuing Dir','Vehicle Going Dir']
df[columns_to_fill_unknown] = df[columns_to_fill_unknown].fillna('Unknown')

In [ ]:
df = df[df['Vehicle Damage Extent'] != 'UNKNOWN']
df = df[df['Vehicle Damage Extent'] != 'OTHER']
df.dropna(subset=['Vehicle Damage Extent'], inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
df['Vehicle Damage Extent'].value_counts().sort_index().plot(kind='bar', color='skyblue')
plt.title('Distribution of Categories')
plt.xlabel('Category')
plt.ylabel('Count')
plt.show()

In [ ]:
df.dtypes

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for column in df.select_dtypes(include=['object']).columns: # convert all object columns to integer
    df[column] = le.fit_transform(df[column])
df

In [ ]:
df.dtypes

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap = 'coolwarm') # draw the correlation graph

In [ ]:
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#features_normalized = scaler.fit_transform(df.columns)

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5)  # Adjust the number of clusters as needed
features = df[['Vehicle Year', 'Vehicle Make', 'Speed Limit']]  # Replace with your selected features
kmeans.fit(features)
df['Cluster'] = kmeans.labels_

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
# Step 2: Prepare the dataset for Logistic Regression
# 'target' should be the column name of your target variable
X = df.drop('Vehicle Damage Extent', axis=1)
y = df['Vehicle Damage Extent']

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 4: Train Logistic Regression Model
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Step 5: Evaluate the Model
predictions = log_reg.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the classifier to the training data
rf_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf_classifier.predict(X_test)

# Evaluate the classifier
predictions = rf_classifier.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
nb_classifier = GaussianNB()

# Fit the classifier to the training data
nb_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = nb_classifier.predict(X_test)

# Evaluate the classifier
predictions = nb_classifier.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
sgd_classifier = SGDClassifier(loss='hinge', max_iter=1000, random_state=42)

# Fit the classifier to the training data
sgd_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = sgd_classifier.predict(X_test)

# Evaluate the classifier
predictions = sgd_classifier.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

In [ ]:
voting_classifier = VotingClassifier(
    estimators=[('lr', log_reg), ('rf', rf_classifier), ('nb', nb_classifier),('sgd',sgd_classifier)],
    voting='hard'  # Use hard voting
)

# Fit the voting classifier to the training data
voting_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = voting_classifier.predict(X_test)

# Evaluate the voting classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
adaboost_classifier = AdaBoostClassifier(
    base_estimator = rf_classifier,  # Use the weak learner as the base estimator
    n_estimators=50,  # Number of weak learners (you can adjust this)
    random_state=42
)

# Fit the AdaBoost classifier to the training data
adaboost_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = adaboost_classifier.predict(X_test)

# Evaluate the AdaBoost classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [ ]:
X = df.drop('Vehicle Damage Extent', axis=1)
y = df['Vehicle Damage Extent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
clf = DecisionTreeClassifier(random_state=42)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))
f1 = f1_score(y_test, y_pred, average = 'weighted')
print(f1)

In [ ]:
feature_importances = clf.feature_importances_

for feature, importance in zip(X.columns, feature_importances):
    print(f'{feature}: {importance:.4f}')